Importanto bibliotecas

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

Inicializando parâmetros

In [ ]:

def init_params(layers_dim:list):
   weight_list = []
   for i in range(1, len(layers_dim)):
     weight_list.append(np.random.uniform(low=-1,high=1,size=(layers_dim[i-1]+1,layers_dim[i])))
    # weight_list.append(np.random.normal(size=(layers_dim[i-1]+1,layers_dim[i])))
   return weight_list

Forward propagation

In [ ]:
def sigmoide(z):
  return 1.0/(1.0 + np.exp(-z))

def sigmoide_linha(z):
  return sigmoide(z)*(1 - sigmoide(z))

def forward_propagation(weight_list,x):
  signal_list = []
  output_list = []

  signal_list.append(np.dot(weight_list[0].T, x))
  output_list.append(np.insert(sigmoide(signal_list[0]),0,1.0,0))

  for i in range(1,len(weight_list)-1):
    signal_list.append(np.dot(weight_list[i].T, output_list[i-1]))
    output_list.append(np.insert(sigmoide(signal_list[i]),0,1.0,0))

  signal_list.append(np.dot(weight_list[-1].T, output_list[-1]))
  output_list.append(signal_list[-1])

  return signal_list,output_list

Backward_propagation

In [ ]:
def backward_propagation(signal_list, output_list, weight_list, x, y):
  grad_list = []
  sense_L = 2*(output_list[-1] - y)
  grad_list.append(np.dot(output_list[-2],sense_L.T))
  e = np.sum((output_list[-1] - y)**2)

  for i in range(len(weight_list)-2, 0, -1):
    sense_L = sigmoide_linha(signal_list[i])*(np.delete(np.dot(weight_list[i+1], sense_L), 0, 0))
    grad_list.append(np.dot(output_list[i-1], sense_L.T))

  sense_L = sigmoide_linha(signal_list[0])*(np.delete(np.dot(weight_list[1], sense_L), 0, 0))
  grad_list.append(np.dot(x,sense_L.T))
  grad_list.reverse()
  return grad_list, e

In [ ]:
def update_params(weight_list, grad_list, eta):
  for i in range(len(weight_list)):
    norm_Wi = np.linalg.norm(weight_list[i])
    if(norm_Wi!=0.0):
      weight_list[i] -= eta * grad_list[i]/norm_Wi
    else:
      weight_list[i] -= eta * grad_list[i]
  return weight_list

Metrics

In [ ]:
def get_prediction(y_n):
  return np.argmax(y_n)

def get_accuracy(weight_list, X_test, Y_test):
  cont = 0 ; m = len(X_test)
  for i in range(m):
    signal_list, output_list = forward_propagation(weight_list, X_test[i])
    y_hat = get_prediction(output_list[-1])
    y = get_prediction(Y_test[i])
    if y_hat == y:
      cont += 1
  return 100 * cont/m

In [ ]:
from tensorflow.python.ops.custom_gradient import backprop
from tqdm import tqdm
def fit(X, Y, X_test, Y_test, layers_shape, learning_rate, batch, epochs_batch):
  weight_list = init_params(layers_shape)
  error_list = []
  accuracy_list = []
  G_list = []
  for k in range(len(weight_list)):
        G_list.append(1)
  for i in range(epochs_batch):
    error = 0.0
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    X_rand = X[indices]
    Y_rand = Y[indices]
    for j in tqdm(range(0, X.shape[0], batch)):
      for k in range(len(weight_list)):
        G_list[k] = (0.0*weight_list[k])

      for k in range(j,j+batch):
        signal_list, output_list = forward_propagation(weight_list,X[j])
        grad_list_aux, e = backward_propagation(signal_list, output_list, weight_list, X[j], Y[j])
        error+=e
        for l in range(len(G_list)):
          G_list[l] = G_list[l]+ grad_list_aux[l]

      for k in range(len(G_list)):
        G_list[k] = G_list[k]/batch
      weight_list = update_params(weight_list, G_list, learning_rate)
    error_list.append(error/X.shape[0])
    accuracy_test = get_accuracy(weight_list, X_test, Y_test)
    accuracy_train = get_accuracy(weight_list, X, Y)
    accuracy_list.append(accuracy_test)
    print(f"Epoch {i}: accuracy test={accuracy_test}%; accuracy train={accuracy_train}%; error={error_list[-1]}")

  return weight_list, error_list, accuracy_list

In [ ]:
# Load the MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize pixel values to have mean 0 and standard deviation 1
#train_images = (train_images / 255.0 - 0.5) / 0.5
train_images = (train_images / 255.0)
#test_images = (test_images / 255.0 - 0.5) / 0.5
test_images = (test_images / 255.0)

# Reshape images
train_images = train_images.reshape(-1, 28 * 28, 1)
test_images = test_images.reshape(-1, 28 * 28, 1)

# Add a bias term (ones column) to the input data
train_images = np.concatenate([np.ones((train_images.shape[0], 1, 1)), train_images], axis=1)
test_images = np.concatenate([np.ones((test_images.shape[0], 1, 1)), test_images], axis=1)

# One-hot encode the labels
num_classes = 10  # Number of classes (digits 0-9)
train_labels_one_hot = keras.utils.to_categorical(train_labels, num_classes)
test_labels_one_hot = keras.utils.to_categorical(test_labels, num_classes)

# Reshape the one-hot encoded labels to (num_samples, num_classes, 1)
train_labels_one_hot = train_labels_one_hot.reshape(-1, num_classes, 1)
test_labels_one_hot = test_labels_one_hot.reshape(-1, num_classes, 1)

11490434/11490434 [==============================] - 2s 0us/step


In [ ]:
weight_list, error_list, accuracy_list = fit(train_images, train_labels_one_hot, test_images, test_labels_one_hot, [28*28 , 128, 64, 10], 0.1, 1, 100)

100%|██████████| 60000/60000 [01:39<00:00, 601.86it/s]


Epoch 0: accuracy test=47.36%; accuracy train=47.955%; error=0.9202024171629608


100%|██████████| 60000/60000 [01:34<00:00, 636.01it/s]


Epoch 1: accuracy test=60.63%; accuracy train=60.915%; error=0.6070811915391202


100%|██████████| 60000/60000 [01:33<00:00, 640.40it/s]


...

Epoch 7: accuracy test=84.88%; accuracy train=84.39666666666666%; error=0.29926771101822475


100%|██████████| 60000/60000 [01:31<00:00, 654.58it/s]


Epoch 8: accuracy test=85.84%; accuracy train=85.52%; error=0.28484881591015426


100%|██████████| 60000/60000 [01:29<00:00, 668.52it/s]


KeyboardInterrupt: 